# Creating plots for movie .gif

This notebook creates maps of the global earthquakes which for each year. It creates these in a loop which titles eachplot by year and then saves each plot as a .png file

In [39]:
source('initialize_data.R', echo = FALSE)

In [40]:
suppressWarnings(suppressMessages(load_libraries()))

[1] "libraries loaded successfully"


Earthquake data must be loaded.

In [41]:
#Loading data and initializing dataframes

path <- "~/jupyter/cs2019_Group11/GroupProducts/data" #Explicit location of data

df <- loadFiles(path) %>%
    reqCols %>%
    parseDt('time')

dfsp <- tospdf(df)

Files in "~/jupyter/cs2019_Group11/GroupProducts/data" loaded.
Subset Completed.
Parsed column: time.
spatial df created


And checked.....    

In [42]:
t(head(df))
str(df)
print(object.size(df), units="Mb")

time,1999-07-01 23:29:27,1999-07-01 22:10:51,1999-07-01 21:42:41,1999-07-01 20:50:11,1999-07-01 19:52:19,1999-07-01 19:52:12
latitude,45.4500,46.7105,51.9950,-31.7260,59.8900,36.0500
longitude,3.790,-122.778,177.972,-72.053,-153.560,31.166
depth,2.000,4.613,106.800,26.100,136.300,30.200
mag,2.6,3.1,4.2,3.9,3.1,3.8
id,usp0009awp,uw10474128,usp0009awk,usp0009awh,usp0009awf,usp0009awe
updated,2014-11-07 01:08:01,2016-07-23 18:59:57,2014-11-07 01:08:01,2014-11-07 01:08:01,2014-11-07 01:08:01,2014-11-07 01:08:01
time_Date,1999-07-01,1999-07-01,1999-07-01,1999-07-01,1999-07-01,1999-07-01
time_Time,18:29:27,17:10:51,16:42:41,15:50:11,14:52:19,14:52:12
time_Year,1999,1999,1999,1999,1999,1999


Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	511837 obs. of  10 variables:
 $ time     : POSIXct, format: "1999-07-01 23:29:27" "1999-07-01 22:10:51" ...
 $ latitude : num  45.5 46.7 52 -31.7 59.9 ...
 $ longitude: num  3.79 -122.78 177.97 -72.05 -153.56 ...
 $ depth    : num  2 4.61 106.8 26.1 136.3 ...
 $ mag      : num  2.6 3.1 4.2 3.9 3.1 3.8 4.5 4.5 4.8 4.8 ...
 $ id       : chr  "usp0009awp" "uw10474128" "usp0009awk" "usp0009awh" ...
 $ updated  : POSIXct, format: "2014-11-07 01:08:01" "2016-07-23 18:59:57" ...
 $ time_Date: Date, format: "1999-07-01" "1999-07-01" ...
 $ time_Time: 'ITime' int  18:29:27 17:10:51 16:42:41 15:50:11 14:52:19 14:52:12 14:45:44 14:26:28 14:03:06 13:56:04 ...
 $ time_Year: num  1999 1999 1999 1999 1999 ...
68.3 Mb


Tectonic plate files and the base world map loaded.

In [43]:
base_world <- base_world()

tectonicdata = "~/jupyter/cs2019_Group11/GroupProducts/plate-boundaries.kmz"
tectonicFeatures <- readOGR(tectonicdata)

Warning message in readOGR(tectonicdata):
“First layer Plate Interface read; multiple layers present in
/dsa/home/jaw56m/jupyter/cs2019_Group11/GroupProducts/plate-boundaries.kmz, check layers with ogrListLayers()”

OGR data source with driver: LIBKML 
Source: "/dsa/home/jaw56m/jupyter/cs2019_Group11/GroupProducts/plate-boundaries.kmz", layer: "Plate Interface"
with 1175 features
It has 12 fields


Warning message in readOGR(tectonicdata):
“Z-dimension discarded”

## Loop to plot earthquakes per year

The following loop plots earthquakes per a year, per a plot. Year data is subset eithin a loop to get each year's data. Plot are also named based on the year they come from. All plots are saved to a list. 

In [44]:
##Jessica
options(repr.plot.width=44, repr.plot.height=40)

plot_list = list()
for (n in 1:20){
    i = 1998+n
world_earthquakes = base_world + geom_point(data= subset(df, df$time_Year == i),aes(x=longitude, y=latitude, color= mag), alpha=I(0.1)) 
# Here I take the map I've made and add the locations for each earthquake and subset the years

map = world_earthquakes + 
    geom_path(data = tectonicFeatures, 
          aes(x = long, y = lat, group = group),
          color = 'gray', size = .4)+
    #now I facet by year inorder to make changes more clear

    theme(strip.background = element_rect(colour="black", fill="white", 
                                       size=1.5, linetype="blank"),strip.text = element_text(size=30),
     legend.text=element_text(size=26), legend.title=element_text(size=40))+
    #I want to change the default title displays for the facets to something cleaner

    scale_color_distiller(palette = "RdPu") + 
    # I change the default color to pink as it contrasts well with the green continents 

    labs(color = "Magnitude")+
    # I rename the legend title to the full name

    ggtitle(paste("Earthquakes >= 2.5 Magitude per Year", i)) +
    # I give a proper title to include the correct years 

    theme(plot.title = element_text(hjust = 0.5, size = 35, face = "bold"))
    # I make the title larger and adjust it to make it more visible 
    plot_list[[n]] = map
}

The following loop takes each plot in the list and converts it to a .png file and saves it including the year which is plotted for each plot in the name. 

In [45]:
for (i in 1:20) {
    n = 1998+ i #want the year to be in the name 
    file_name = paste("earthquake", n, ".png", sep="")
    png(file_name, width = 1200, height = 800)
    print(plot_list[[i]])
    dev.off()
}

From here the a python script will be ran on the output to convert these .png files into a single .gif file.